### CARGA DE LIBRERIAS

In [60]:
import pandas as pd
import numpy as np

import os
from datetime import datetime
from openpyxl import load_workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.utils import get_column_letter

from time import sleep
from rapidfuzz import fuzz, process



### PLANTILLAS


In [61]:
import os
import pandas as pd
import sys

# Detectar la ruta base desde donde se ejecuta (compatible con .exe y script normal)
if getattr(sys, 'frozen', False):
    base_path = sys._MEIPASS  # cuando está empaquetado con pyinstaller
else:
    try:
        # Cuando se ejecuta como script .py normal
        base_path = os.path.dirname(os.path.abspath(__file__))
    except NameError:
        # Cuando se ejecuta en notebook o intérprete interactivo (sin __file__)
        base_path = os.getcwd()

# Construir la ruta al archivo Excel
Path_File = os.path.join(base_path, 'Plantilla_Quotation.xlsx')

# Cargar el Excel y transformar a mayúsculas
Template = pd.read_excel(Path_File, skiprows=1, header=0)
Template = Template.map(lambda x: x.upper() if isinstance(x, str) else x)

In [62]:
zip_cols = ['ZIP Code', 'ZIP Code.1']
Template[zip_cols] = Template[zip_cols].apply(pd.to_numeric, errors='coerce').astype('Int64')

In [63]:
Template["Maritimo"] = Template["POL"].notna() & Template["POD"].notna()
# Template.head(4)

In [64]:
# Dividimos el Dataset en 2 opciones (flujo unicamente terrestre y flujo mixto (maritimo))
Template_Terrestre = Template[Template["Maritimo"] == False]
Template_Maritimo = Template[Template["Maritimo"] == True]
# Template_Maritimo

### DATABASES

In [65]:
# INLAND RATES

# Path_File =  r"Z:\Excellence\03- Data Governance\Digital\01- Projects\20 - Quotation Tool\cifrados Overseas-Inland.xlsx"
Path_File = os.path.join(base_path, 'cifrados Overseas-Inland.xlsx')
CifradosInland = pd.read_excel(Path_File, sheet_name="INLAND")
CifradosInland["Picking_Country"] = CifradosInland["Picking_Country"].str.upper()
CifradosInland["Delivery_Country"] = CifradosInland["Delivery_Country"].str.upper()
CifradosInland["Origin_City"] = CifradosInland["Origin_City"].str.upper()
CifradosInland["Destination_City"] = CifradosInland["Destination_City"].str.upper()

In [66]:
# Path_File =  r"Z:\Excellence\03- Data Governance\Digital\01- Projects\20 - Quotation Tool\Distances_Costs Country_Port.xlsx"
Path_File = os.path.join(base_path, 'Distances_Costs Country_Port.xlsx')
RatesInland = pd.read_excel(Path_File)
# CifradosInland.head(3)


In [67]:
# OVERSEAS RATES
# Path_File = r"Z:\Excellence\03- Data Governance\Digital\01- Projects\20 - Quotation Tool\RATES_04_2025.xlsx"
Path_File = os.path.join(base_path, 'RATES_04_2025.xlsx')
CifradosOverseas = pd.read_excel(Path_File, sheet_name="MAIN PORTS")
PlantaPuerto = pd.read_excel(Path_File, sheet_name="HORSE-PUERTO")
PlantaPuerto["ZIP Code"] = PlantaPuerto["ZIP Code"].apply(lambda x: int(x) if pd.notnull(x) else pd.NA).astype("Int64")


In [68]:
# EMBALAJES 
# Path_File =  r"Z:\Excellence\03- Data Governance\Digital\01- Projects\20 - Quotation Tool\Reduced Packaging.xlsx"

Path_File = os.path.join(base_path, 'Reduced Packaging.xlsx')
Mapping_Motores = pd.read_excel(Path_File)
Mapping_Motores_reducido = Mapping_Motores[["Reference","Packaging Code","Qté / UC","Part Weight (kg)","Weight EMPTY (kg)","Hauteur hors tout","Largeur hors tout","Longueur hors tout"]]

Mapping_Motores_reducido = Mapping_Motores_reducido.rename(columns={
    "Hauteur hors tout": "Altura",
    "Largeur hors tout": "Anchura",
    "Longueur hors tout": "Largo"
})
# Quitamos duplicados en base a Reference y Packaging Code

Mapping_Motores_reducido = Mapping_Motores_reducido.drop_duplicates(subset=["Reference", "Packaging Code"])
Mapping_Motores_reducido.columns

Index(['Reference', 'Packaging Code', 'Qté / UC', 'Part Weight (kg)',
       'Weight EMPTY (kg)', 'Altura', 'Anchura', 'Largo'],
      dtype='object')

In [69]:
Path_File = os.path.join(base_path, 'LEAD_TIME_FINAL.xlsx')
VTT_Final = pd.read_excel(Path_File)

In [70]:
Interes_Financiero = 0.078

### MARÍTIMO

##### Marítimo 1

In [71]:
# Diccionario con columnas a renombrar
renombrar_columnas = {
    "Country": "Pais_Origen",
    "ZIP Code": "ZIP Origen",
    "Name" : "Nombre_Origen",
    "City": "City_Origen",
    "Country.1" : "Pais_Destino",
    "ZIP Code.1" : "Zip_Destino",
    "Name.1" : "Name_Destino",
    "City.1" : "City_Destino"
}
Template_Maritimo = Template_Maritimo.rename(columns=renombrar_columnas)


In [72]:
# Si el primero es planta - puerto 

zipcodes_plantas = PlantaPuerto["ZIP Code"].dropna().unique().tolist()


# Dividir Template_Maritimo en dos partes
Template_Maritimo_Planta = Template_Maritimo[Template_Maritimo["ZIP Origen"].isin(zipcodes_plantas)]
Template_Maritimo_Proveedor= Template_Maritimo[~Template_Maritimo["ZIP Origen"].isin(zipcodes_plantas)]

# MERGE con PlantaPuerto para códigos especiales usando dos columnas
Template_Maritimo_Planta = Template_Maritimo_Planta.merge(
    PlantaPuerto,
    left_on=["ZIP Origen", "POL"],
    right_on=["ZIP Code", "Port"],
    how="inner"
)

# Crear la nueva columna 'Rate_maritimo_1' como la multiplicación
Template_Maritimo_Planta["Rate_maritimo_1"] = (
    Template_Maritimo_Planta["Distance (km)"] * Template_Maritimo_Planta["Eur/km"]
).round(2)
# Eliminar columnas no deseadas
columns_to_drop = ['Factory','City_y', 'Country Code', 'Country_y', 'Port', 'Port Zip', 'Distance (km)','Maritimo', 'Eur/km',"ZIP Code", "City", "Country"]
Template_Maritimo_Planta = Template_Maritimo_Planta.drop(columns=columns_to_drop, errors="ignore")



In [73]:
Template_Maritimo_Proveedor["merge_id"] = Template_Maritimo_Proveedor.index

Template_Maritimo_Proveedor = Template_Maritimo_Proveedor.merge(
    RatesInland,
    left_on=["City_Origen", "POL"],
    right_on=["CITY-NAME", "Port Name"],
    how="left",
    indicator=True
)

from rapidfuzz import fuzz


def elegir_mejor_fila(grupo):
    origen = grupo["Nombre_Origen"].iloc[0]
    grupo = grupo.copy()
    grupo["sim_score"] = grupo["LEGAL-ENTITY"].fillna("").apply(
        lambda x: fuzz.token_sort_ratio(str(origen), str(x))
    )
    return grupo.sort_values("sim_score", ascending=False).iloc[0]

Template_Maritimo_Proveedor = (
    Template_Maritimo_Proveedor
    .groupby("merge_id", group_keys=False)
    .apply(elegir_mejor_fila)
    .drop(columns=["sim_score", "merge_id"], errors="ignore")
)

Template_Maritimo_Proveedor["Indicador errores"] = Template_Maritimo_Proveedor.apply(
    lambda row: (
        f"Combination '{row['City_Origen']}' + '{row['POL']}' not found in Distances_Costs Country_Port."
        if row["_merge"] != "both" else ""
    ),
    axis=1
)

columns_to_drop = ['Maritimo', 'LEGAL_MATCH', 'MATCH-CODE', 'COUNTRY-CODE',
       'Supplier Country', 'CITY-NAME', 'Port Name', 'Port Country',
       'Distance (km)', 'Fecha', 'Cost', 'PICKING COUNTRY', 'DELIVERY COUNTRY',
       'Cost per Km',  '_merge','LEGAL-ENTITY','merge_id']

Template_Maritimo_Proveedor = Template_Maritimo_Proveedor.drop(columns=columns_to_drop, errors="ignore")

Template_Maritimo_Proveedor = Template_Maritimo_Proveedor.rename(columns={
    'Total Cost': 'Rate_maritimo_1',
})


C:\Users\SANCHEZ-SERRADORMiri\AppData\Local\Temp\ipykernel_17584\3611894513.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(elegir_mejor_fila)


In [74]:
# Juntar de nuevo ambos resultados
Template_Maritimo = pd.concat([Template_Maritimo_Planta, Template_Maritimo_Proveedor], ignore_index=True)


#####  Marítimo 2 overseas

In [75]:
# MERGE con indicador para detectar combinaciones POL + POD no encontradas
Template_Maritimo = Template_Maritimo.merge(
    CifradosOverseas,
    on=["POL", "POD"],
    how="left",
    indicator=True
)

# Crear la columna si no existe
if "Indicador errores" not in Template_Maritimo.columns:
    Template_Maritimo["Indicador errores"] = ""

# Concatenar mensaje si no se encontró la combinación
Template_Maritimo["Indicador errores"] = Template_Maritimo.apply(
    lambda row: (
        str(row["Indicador errores"]) + 
        (f"Combination '{row['POL']}' + '{row['POD']}' not found in RATES_04_2025 reference mapping. " 
         if row["_merge"] != "both" else "")
    ),
    axis=1
)

# Eliminar columna auxiliar
Template_Maritimo.drop(columns=["_merge"], inplace=True)

# Asegurar que los valores de "Rate 40ft all-in" sean numéricos
CifradosOverseas["Rate 40ft all-in"] = pd.to_numeric(CifradosOverseas["Rate 40ft all-in"], errors='coerce')


columns_to_drop = ['ID_GROUP',
       'PLANT TO PORTX', 'Country', 'POL ZIP', 'Country2', 'POD ZIP',
       'Rate 20ft all-in', 'Rate 20ft all-in % Renault',
       'Rate 40ft all-in % Renault', 'x', 'c']

# Eliminar las columnas
Template_Maritimo = Template_Maritimo.drop(columns=columns_to_drop)

Template_Maritimo = Template_Maritimo.rename(columns={
    'Rate 40ft all-in': 'Rate_maritimo_2'
    # , "Country_x" :"Country"
    
})

In [76]:
Template_Maritimo

,Flow,Part Number (PN),Part Designation,Nombre_Origen,Pais_Origen,City_Origen,ZIP Origen,Name_Destino,Pais_Destino,City_Destino,Zip_Destino,Anual Needs,Daily Need,Unit cost (€),Packaging Code,POL,POD,Rate_maritimo_1,Indicador errores,Rate_maritimo_2
0,NaN,110103878R,CARTER CYLINDRE ASS,HORSE MOTORES,ES,VALLADOLID,47008,HORSE BRASIL,BR,CURITIBA,<NA>,2000,9.000000,NaN,MFM---0843,ESVLC,BRPNG,726.8,nan,226
1,NaN,110103478R,CYLINDER BLOCK,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,322000,300000,1363.636364,116.4,MFM---7923,ESVLC,CNNBO,726.8,nan,450
2,NaN,8201755965,CARTER HEAD,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,322000,300000,1363.636364,220.5,MFM---7923,ESVLC,CNNBO,726.8,nan,450
3,NaN,122018605R,CRACKSHAFT,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,322000,300000,1363.636364,57.2,MFM---7923,ESVLC,CNNBO,726.8,nan,450
4,NaN,130013924R,INTAKE CAMSHAFT,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,322000,300000,1363.636364,17.8,MFM---7923,ESVLC,CNNBO,726.8,nan,450
5,NaN,130011303R,EXHAUST CAMSHAT,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,322000,300000,1363.636364,19.9,MFM---7923,ESVLC,CNNBO,726.8,nan,450
6,NaN,121002471R,COMROD,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,322000,1200000,5454.545455,3.8,MFM---7923,ESVLC,CNNBO,726.8,nan,450
7,NaN,100016158R,ENGINE,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,322000,300000,1364.000000,3000.0,MFM---7923,ESVLC,CNNBO,726.8,nan,450


##### Marítimo 3 

In [77]:
# Si el ultimo es puerto - planta 

zipcodes_plantas = PlantaPuerto["ZIP Code"].dropna().unique().tolist()
 

# Dividir Template_Maritimo en dos partes
Template_Maritimo_Planta_Destino = Template_Maritimo[Template_Maritimo["Zip_Destino"].isin(zipcodes_plantas)]
Template_Maritimo_Proveedor_Destino= Template_Maritimo[~Template_Maritimo["Zip_Destino"].isin(zipcodes_plantas)]

# MERGE con PlantaPuerto para códigos especiales usando dos columnas
Template_Maritimo_Planta_Destino = Template_Maritimo_Planta_Destino.merge(
    PlantaPuerto,
    left_on=["POD", "Zip_Destino"],
    right_on=["Port", "ZIP Code"],
    how="inner"
)

# Crear la nueva columna 'Rate_maritimo_1' como la multiplicación
Template_Maritimo_Planta_Destino["Rate_maritimo_3"] = (
    Template_Maritimo_Planta_Destino["Distance (km)"] * Template_Maritimo_Planta_Destino["Eur/km"]
).round(2)
# Eliminar columnas no deseadas
columns_to_drop = ['Factory','City_y', 'Country Code', 'Country_y', 'Port', 'Port Zip', 'Distance (km)', 'Eur/km',"ZIP Code", "City", "Country"]
Template_Maritimo_Planta_Destino = Template_Maritimo_Planta_Destino.drop(columns=columns_to_drop, errors="ignore")


In [78]:
Template_Maritimo_Proveedor_Destino.columns

Index(['Flow', 'Part Number (PN)', 'Part Designation', 'Nombre_Origen',
       'Pais_Origen', 'City_Origen', 'ZIP Origen', 'Name_Destino',
       'Pais_Destino', 'City_Destino', 'Zip_Destino', 'Anual Needs',
       'Daily Need', 'Unit cost (€)', 'Packaging Code', 'POL', 'POD',
       'Rate_maritimo_1', 'Indicador errores', 'Rate_maritimo_2'],
      dtype='object')

In [79]:
Template_Maritimo_Proveedor_Destino["merge_id"] = Template_Maritimo_Proveedor_Destino.index
Template_Maritimo_Proveedor_Destino = Template_Maritimo_Proveedor_Destino.merge(
    RatesInland,
    left_on=["City_Destino", "POD"],
    right_on=["CITY-NAME", "Port Name"],
    how="left",
    indicator=True
)
from rapidfuzz import fuzz

def elegir_mejor_fila(grupo):
    origen = grupo["Name_Destino"].iloc[0]
    grupo = grupo.copy()
    grupo["sim_score"] = grupo["LEGAL-ENTITY"].fillna("").apply(
        lambda x: fuzz.token_sort_ratio(str(origen), str(x))
    )
    return grupo.sort_values("sim_score", ascending=False).iloc[0]

Template_Maritimo_Proveedor_Destino = (
    Template_Maritimo_Proveedor_Destino
    .groupby("merge_id", group_keys=False)
    .apply(elegir_mejor_fila)
    .drop(columns=["sim_score", "merge_id"], errors="ignore")
)

# Crear la columna si no existe
if "Indicador errores" not in Template_Maritimo_Proveedor_Destino.columns:
    Template_Maritimo_Proveedor_Destino["Indicador errores"] = ""

Template_Maritimo_Proveedor_Destino["Indicador errores"] = Template_Maritimo_Proveedor_Destino.apply(
    lambda row: (
        str(row["Indicador errores"]) + 
        (f"Combination '{row['City_Destino']}' + '{row['POD']}' not found in Distances_Costs Country_Port. "
         if row["_merge"] != "both" else "")
    ),
    axis=1
)


columns_to_drop = ['Maritimo', 'LEGAL_MATCH', 'MATCH-CODE', 'COUNTRY-CODE',
       'Supplier Country', 'CITY-NAME', 'Port Name', 'Port Country',
       'Distance (km)', 'Fecha', 'Cost', 'PICKING COUNTRY', 'DELIVERY COUNTRY',
       'Cost per Km',  '_merge','LEGAL-ENTITY','merge_id']
Template_Maritimo_Proveedor_Destino = Template_Maritimo_Proveedor_Destino.drop(columns=columns_to_drop, errors="ignore")
Template_Maritimo_Proveedor_Destino = Template_Maritimo_Proveedor_Destino.rename(columns={
    'Total Cost': 'Rate_maritimo_3',
})

C:\Users\SANCHEZ-SERRADORMiri\AppData\Local\Temp\ipykernel_17584\2957372107.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(elegir_mejor_fila)


In [80]:
# Juntar de nuevo ambos resultados
Template_Maritimo = pd.concat([Template_Maritimo_Planta_Destino, Template_Maritimo_Proveedor_Destino], ignore_index=True)

In [81]:
Template_Maritimo

,Flow,Part Number (PN),Part Designation,Nombre_Origen,Pais_Origen,City_Origen,ZIP Origen,Name_Destino,Pais_Destino,City_Destino,...,Anual Needs,Daily Need,Unit cost (€),Packaging Code,POL,POD,Rate_maritimo_1,Indicador errores,Rate_maritimo_2,Rate_maritimo_3
0,NaN,110103878R,CARTER CYLINDRE ASS,HORSE MOTORES,ES,VALLADOLID,47008,HORSE BRASIL,BR,CURITIBA,...,2000,9.000000,NaN,MFM---0843,ESVLC,BRPNG,726.8,nan,226,0.0
1,NaN,110103478R,CYLINDER BLOCK,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,...,300000,1363.636364,116.4,MFM---7923,ESVLC,CNNBO,726.8,nanCombination 'ZHEIJANG' + 'CNNBO' not found ...,450,NaN
2,NaN,8201755965,CARTER HEAD,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,...,300000,1363.636364,220.5,MFM---7923,ESVLC,CNNBO,726.8,nanCombination 'ZHEIJANG' + 'CNNBO' not found ...,450,NaN
3,NaN,122018605R,CRACKSHAFT,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,...,300000,1363.636364,57.2,MFM---7923,ESVLC,CNNBO,726.8,nanCombination 'ZHEIJANG' + 'CNNBO' not found ...,450,NaN
4,NaN,130013924R,INTAKE CAMSHAFT,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,...,300000,1363.636364,17.8,MFM---7923,ESVLC,CNNBO,726.8,nanCombination 'ZHEIJANG' + 'CNNBO' not found ...,450,NaN
5,NaN,130011303R,EXHAUST CAMSHAT,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,...,300000,1363.636364,19.9,MFM---7923,ESVLC,CNNBO,726.8,nanCombination 'ZHEIJANG' + 'CNNBO' not found ...,450,NaN
6,NaN,121002471R,COMROD,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,...,1200000,5454.545455,3.8,MFM---7923,ESVLC,CNNBO,726.8,nanCombination 'ZHEIJANG' + 'CNNBO' not found ...,450,NaN
7,NaN,100016158R,ENGINE,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,...,300000,1364.000000,3000.0,MFM---7923,ESVLC,CNNBO,726.8,nanCombination 'ZHEIJANG' + 'CNNBO' not found ...,450,NaN


### Embalage maritimo

In [82]:
Template_Maritimo = Template_Maritimo.merge(
    Mapping_Motores_reducido,
    left_on=["Part Number (PN)", "Packaging Code"],
    right_on=["Reference", "Packaging Code"],
    how="left",
    indicator=True  # crea la columna "_merge"
)

# Valores únicos de referencia y packaging en el mapping
referencias_validas = set(Mapping_Motores_reducido["Reference"].dropna().unique())
packaging_validos = set(Mapping_Motores_reducido["Packaging Code"].dropna().unique())


# Función para generar y concatenar errores
def obtener_error(row):
    errores_nuevos = []

    ref = row["Part Number (PN)"]
    pack = row["Packaging Code"]

    if pd.isna(ref) or ref not in referencias_validas:
        errores_nuevos.append(f"Reference '{ref}' not found in Reduced Packaging.")

    if pd.isna(pack) or pack not in packaging_validos:
        errores_nuevos.append(f"Packaging Code '{pack}' not found in Reduced Packaging.")

    if not errores_nuevos and row["_merge"] != "both":
        errores_nuevos.append(f"Combination '{ref}' + '{pack}' not found in Reduced Packaging.")

    # Concatenar con errores anteriores, si los hay
    errores_previos = row["Indicador errores"].strip()
    if errores_nuevos:
        if errores_previos:
            return errores_previos + " | " + " ".join(errores_nuevos)
        else:
            return " ".join(errores_nuevos)
    else:
        return errores_previos

# Aplicamos la función
Template_Maritimo["Indicador errores"] = Template_Maritimo.apply(obtener_error, axis=1)


# Eliminar columnas que no quieras
Template_Maritimo = Template_Maritimo.drop(columns=["Reference", "_merge"])

In [83]:
Template_Maritimo

,Flow,Part Number (PN),Part Designation,Nombre_Origen,Pais_Origen,City_Origen,ZIP Origen,Name_Destino,Pais_Destino,City_Destino,...,Rate_maritimo_1,Indicador errores,Rate_maritimo_2,Rate_maritimo_3,Qté / UC,Part Weight (kg),Weight EMPTY (kg),Altura,Anchura,Largo
0,NaN,110103878R,CARTER CYLINDRE ASS,HORSE MOTORES,ES,VALLADOLID,47008,HORSE BRASIL,BR,CURITIBA,...,726.8,nan,226,0.0,18.0,13.261,110.0,930.0,1000.0,1200.0
1,NaN,110103478R,CYLINDER BLOCK,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,...,726.8,nanCombination 'ZHEIJANG' + 'CNNBO' not found ...,450,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,8201755965,CARTER HEAD,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,...,726.8,nanCombination 'ZHEIJANG' + 'CNNBO' not found ...,450,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,122018605R,CRACKSHAFT,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,...,726.8,nanCombination 'ZHEIJANG' + 'CNNBO' not found ...,450,NaN,70.0,11.430,157.0,930.0,1000.0,1200.0
4,NaN,130013924R,INTAKE CAMSHAFT,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,...,726.8,nanCombination 'ZHEIJANG' + 'CNNBO' not found ...,450,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,130011303R,EXHAUST CAMSHAT,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,...,726.8,nanCombination 'ZHEIJANG' + 'CNNBO' not found ...,450,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,121002471R,COMROD,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,...,726.8,nanCombination 'ZHEIJANG' + 'CNNBO' not found ...,450,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,100016158R,ENGINE,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,...,726.8,nanCombination 'ZHEIJANG' + 'CNNBO' not found ...,450,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### CALCULO DE COLUMNAS 

In [84]:
# Packaging Volume
Template_Maritimo["Packaging Volume"] = (Template_Maritimo["Largo"] * Template_Maritimo["Anchura"] * Template_Maritimo["Altura"]) / 1_000_000_000

# Part volume
Template_Maritimo["Part volume"] = (Template_Maritimo["Packaging Volume"] / Template_Maritimo["Qté / UC"])

In [85]:
Filling_Weight = 24750  # kg peso máximo
#Filling A volumen 
Filling_Rate_Max_Terrestre = 62


Template_Maritimo["Total_Weight"] = (
    Template_Maritimo["Qté / UC"] * Template_Maritimo["Part Weight (kg)"]
    + Template_Maritimo["Weight EMPTY (kg)"]
)

Template_Maritimo["Saturation"] = Template_Maritimo.apply(
    lambda row: "" if pd.isna(row["Total_Weight"]) 
    else "V" if row["Total_Weight"] < Filling_Weight 
    else "W",
    axis=1
)

def calcular_costes_logistica(df, filling_rate_max, filling_weight):
    # Aseguramos que los parámetros sean flotantes
    filling_rate_max = float(filling_rate_max)
    filling_weight = float(filling_weight)

    # Convertimos las columnas de tasas a numérico, si ya son nan no las tocará
    df['Rate_maritimo_1'] = pd.to_numeric(df['Rate_maritimo_1'], errors='coerce')
    df['Rate_maritimo_2'] = pd.to_numeric(df['Rate_maritimo_2'], errors='coerce')
    df['Rate_maritimo_3'] = pd.to_numeric(df['Rate_maritimo_3'], errors='coerce')
    df['Part volume'] = pd.to_numeric(df['Part volume'], errors='coerce')

    # Creamos las columnas vacías por defecto
    df["Plant to POL €/m3"] = None
    df["POL > POD €/m3 40ft"] = None
    df["POD > Plant €/m3"] = None
    df["Plant to POL €/part"] = None
    df["POL > POD €/part"] = None
    df["POD to Plant €/part"] = None
    df["LOG €/Part"] = None
    df["LOG €/m3"] = None

    # Condición cuando la columna "Saturation" es "V" o "W"
    mask_v = df['Saturation'] == 'V'
    mask_w = df['Saturation'] == 'W'

    # Si "Saturation" es 'V', usamos Filling_Rate_Max
    df.loc[mask_v, "Plant to POL €/m3"] = df.loc[mask_v, "Rate_maritimo_1"] / filling_rate_max
    df.loc[mask_v, "POL > POD €/m3 40ft"] = df.loc[mask_v, "Rate_maritimo_2"] / filling_rate_max / 1.11
    df.loc[mask_v, "POD > Plant €/m3"] = df.loc[mask_v, "Rate_maritimo_3"] / filling_rate_max

    df.loc[mask_v, "Plant to POL €/part"] = df.loc[mask_v, "Plant to POL €/m3"] * df.loc[mask_v, "Part volume"]
    df.loc[mask_v, "POL > POD €/part"] = df.loc[mask_v, "POL > POD €/m3 40ft"] * df.loc[mask_v, "Part volume"]
    df.loc[mask_v, "POD to Plant €/part"] = df.loc[mask_v, "POD > Plant €/m3"] * df.loc[mask_v, "Part volume"]

    df.loc[mask_v, "LOG €/Part"] = (
        df.loc[mask_v, "Plant to POL €/part"] + 
        df.loc[mask_v, "POL > POD €/part"] + 
        df.loc[mask_v, "POD to Plant €/part"]
    )

    df.loc[mask_v, "LOG €/m3"] = (
        df.loc[mask_v, "Plant to POL €/m3"] + 
        df.loc[mask_v, "POL > POD €/m3 40ft"] + 
        df.loc[mask_v, "POD > Plant €/m3"]
    )

    # Si "Saturation" es 'W', usamos Filling_Weight

    Volumen_maximo = (Filling_Weight / Template_Maritimo["Total_Weight"] ) * Template_Maritimo["Packaging Volume"]

    df.loc[mask_w, "Plant to POL €/m3"] = df.loc[mask_w, "Rate_maritimo_1"] / Volumen_maximo
    df.loc[mask_w, "POL > POD €/m3 40ft"] = df.loc[mask_w, "Rate_maritimo_2"] / Volumen_maximo / 1.11
    df.loc[mask_w, "POD > Plant €/m3"] = df.loc[mask_w, "Rate_maritimo_3"] / Volumen_maximo

    df.loc[mask_w, "Plant to POL €/part"] = df.loc[mask_w, "Plant to POL €/m3"] * df.loc[mask_w, "Part volume"]
    df.loc[mask_w, "POL > POD €/part"] = df.loc[mask_w, "POL > POD €/m3 40ft"] * df.loc[mask_w, "Part volume"]
    df.loc[mask_w, "POD to Plant €/part"] = df.loc[mask_w, "POD > Plant €/m3"] * df.loc[mask_w, "Part volume"]

    df.loc[mask_w, "LOG €/Part"] = (
        df.loc[mask_w, "Plant to POL €/part"] + 
        df.loc[mask_w, "POL > POD €/part"] + 
        df.loc[mask_w, "POD to Plant €/part"]
    )

    df.loc[mask_w, "LOG €/m3"] = (
        df.loc[mask_w, "Plant to POL €/m3"] + 
        df.loc[mask_w, "POL > POD €/m3 40ft"] + 
        df.loc[mask_w, "POD > Plant €/m3"]
    )

    return df

# Si la columna 'Saturation' tiene valores 'V' o 'W', aplicamos la función correspondiente
if 'Saturation' in Template_Maritimo.columns:
    # Si 'Saturation' tiene 'V', usamos Filling_Rate_Max, y si tiene 'W', usamos Volumen_maximo
    Template_Maritimo = calcular_costes_logistica(Template_Maritimo, Filling_Rate_Max_Terrestre, Filling_Weight)



### INCLUIR VTTs

In [86]:
# Hacer el left join para mantener todas las filas de Template_Maritimo
Template_Maritimo = Template_Maritimo.merge(
    VTT_Final,
    left_on=["Pais_Origen", "ZIP Origen", "Pais_Destino", "Zip_Destino"],
    right_on=["Picking_Country", "ZIP Code", "Delivery_Country", "ZIP Code.1"],
    how="left",
    indicator=True  # Nos indica si hizo match o no
)

# Asegurarse de que la columna "Indicador errores" ya existe (o crearla vacía si no)
if "Indicador errores" not in Template_Maritimo.columns:
    Template_Maritimo["Indicador errores"] = ""

# Añadir mensaje solo si no hizo match
Template_Maritimo["Indicador errores"] = Template_Maritimo.apply(
    lambda row: (
        (row["Indicador errores"] + " | " if row["Indicador errores"] else "") +
        f"Route from {row['Pais_Origen']} {row['ZIP Origen']} to {row['Pais_Destino']} {row['Zip_Destino']} not found in VTT"
    ) if row["_merge"] != "both" else row["Indicador errores"],
    axis=1
)

# Eliminar columna de tracking del merge
Template_Maritimo = Template_Maritimo.drop(columns=["_merge"])

Template_Maritimo = Template_Maritimo.drop(columns=['transportType', 'childPlantPublicId', 'Picking_Country',
       'ZIP Code', 'childPlantName', 'parentPlantPublicId', 'Delivery_Country',
       'ZIP Code.1', 'parentPlantName', 'leadTime',
       'nbFlows'])

In [87]:
Template_Maritimo

,Flow,Part Number (PN),Part Designation,Nombre_Origen,Pais_Origen,City_Origen,ZIP Origen,Name_Destino,Pais_Destino,City_Destino,...,Saturation,Plant to POL €/m3,POL > POD €/m3 40ft,POD > Plant €/m3,Plant to POL €/part,POL > POD €/part,POD to Plant €/part,LOG €/Part,LOG €/m3,LeadTime (days)
0,NaN,110103878R,CARTER CYLINDRE ASS,HORSE MOTORES,ES,VALLADOLID,47008,HORSE BRASIL,BR,CURITIBA,...,V,11.722581,3.283929,0.0,0.7268,0.203604,0.0,0.930404,15.00651,NaN
1,NaN,110103478R,CYLINDER BLOCK,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,...,,None,None,None,None,None,None,None,None,NaN
2,NaN,8201755965,CARTER HEAD,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,...,,None,None,None,None,None,None,None,None,NaN
3,NaN,122018605R,CRACKSHAFT,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,...,V,11.722581,6.538797,NaN,0.186891,0.104247,NaN,NaN,NaN,NaN
4,NaN,130013924R,INTAKE CAMSHAFT,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,...,,None,None,None,None,None,None,None,None,NaN
5,NaN,130011303R,EXHAUST CAMSHAT,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,...,,None,None,None,None,None,None,None,None,NaN
6,NaN,121002471R,COMROD,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,...,,None,None,None,None,None,None,None,None,NaN
7,NaN,100016158R,ENGINE,MOTORES ENGINE PLANT,SPAIN,VALLADOLID,47008,YIWU ENGINE PLANT,CHINA,ZHEIJANG,...,,None,None,None,None,None,None,None,None,NaN


In [88]:
Template_Maritimo["Floating Stock €/Part"] = (
    Template_Maritimo["Unit cost (€)"] * Interes_Financiero / 365
) * Template_Maritimo["LeadTime (days)"].replace("?", pd.NA)

In [89]:
Template_Maritimo_limpio = Template_Maritimo.drop_duplicates().copy()

Template_Maritimo_limpio = Template_Maritimo_limpio.drop(columns=['Part Weight (kg)', 'Weight EMPTY (kg)'])

TOTALES

In [90]:
Template_Maritimo_limpio["TOTAL €/Part"] = Template_Maritimo_limpio["LOG €/Part"]+Template_Maritimo_limpio["Unit cost (€)"]+Template_Maritimo_limpio["Floating Stock €/Part"]

Template_Maritimo_limpio["Annual weight"] = Template_Maritimo_limpio["TOTAL €/Part"]* Template_Maritimo_limpio["Anual Needs"]/1000

Template_Maritimo_limpio["FCF Pipe"] = Template_Maritimo_limpio["Daily Need"]*Template_Maritimo_limpio["Unit cost (€)"]*Template_Maritimo_limpio["LeadTime (days)"]/1000

In [91]:
# Limpieza final

def renombrar(col):
    if col.endswith('_x') or col.endswith('_y'):
        return col[:-2]  
    else:
        return col

Template_Maritimo_limpio.columns = [renombrar(col) for col in Template_Maritimo_limpio.columns]

Template_Maritimo_limpio.rename(columns={
    "Rate_maritimo_1": "Rate country origin",
    "Rate_maritimo_2": "Rate seafreight",
    "Rate_maritimo_3": "Rate country destination",
    "Altura":"Height",
    "Anchura": "Width",
    "Largo": "Length"
}, inplace=True)

# Eliminar columnas duplicadas
Template_Maritimo_limpio = Template_Maritimo_limpio.loc[:, ~Template_Maritimo_limpio.columns.duplicated()]




### EXPORT A EXCEL MARITIMO

In [92]:
col_a_mover = "Indicador errores"
columnas = [col_a_mover]+ [col for col in Template_Maritimo_limpio.columns if col != col_a_mover] 
Template_Maritimo_limpio = Template_Maritimo_limpio[columnas]


In [93]:
renombrar_columnas = {
    "Indicador errores": "Error Indicator",
    "Nombre_Origen": "Name of Origin",
    "Pais_Origen" : "Country of Origin",
    "City_Origen": "City of Origin",
    "ZIP Origen" : "Origin ZIP",
    "Name_Destino" : "Name of Destination",
    "Pais_Destino" : "Country of Destination",
    "City_Destino" : "City of Destination",
    "Zip_Destino" : "Destination ZIP"
}
Template_Maritimo_limpio = Template_Maritimo_limpio.rename(columns=renombrar_columnas)

In [94]:
Template_Maritimo_limpio["Error Indicator"] = Template_Maritimo_limpio["Error Indicator"].str.replace(r"nan", "", regex=True)


In [95]:
for col in Template_Maritimo_limpio.columns:
    if col != 'Daily Need':
        try:
            Template_Maritimo_limpio[col] = pd.to_numeric(Template_Maritimo_limpio[col], errors='ignore')
        except:
            pass  # En caso de error, dejar como está

# Redondear columnas numéricas (excepto 'Daily Need') a 2 decimales
cols_to_round = Template_Maritimo_limpio.select_dtypes(include=[np.number]).columns.drop('Daily Need', errors='ignore')
Template_Maritimo_limpio[cols_to_round] = Template_Maritimo_limpio[cols_to_round].round(2)

# Truncar 'Daily Need' sin afectar NaN
Template_Maritimo_limpio['Daily Need'] = Template_Maritimo_limpio['Daily Need'].apply(
    lambda x: int(x) if pd.notna(x) else np.nan
)

C:\Users\SANCHEZ-SERRADORMiri\AppData\Local\Temp\ipykernel_17584\2317680209.py:4: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  Template_Maritimo_limpio[col] = pd.to_numeric(Template_Maritimo_limpio[col], errors='ignore')


In [96]:
# Ruta del directorio

folder_maritimo = "Maritime Tool"
directory = os.path.join(base_path, folder_maritimo)
# Inicializa un contador
counter = 1

# Obtener la fecha actual en formato YYYY-MM-DD
current_date = datetime.now().strftime('%Y-%m-%d')

# Crear el nombre del archivo con la fecha
file_name = f"Maritime_Template_{current_date}_{counter}.xlsx"

# Asegurar que el archivo no existe, incrementar el contador
while os.path.exists(os.path.join(directory, file_name)):
    counter += 1
    file_name = f"Maritime_Template_{current_date}_{counter}.xlsx"

# Ruta completa del archivo
full_path = os.path.join(directory, file_name)

# Guardar el DataFrame como Excel
Template_Maritimo_limpio.to_excel(full_path, index=False, sheet_name="Hoja1")

# --- FORMATEAR COMO TABLA ---
# Cargar el archivo con openpyxl
wb = load_workbook(full_path)
ws = wb["Hoja1"]

# Obtener el rango de la tabla
max_row = ws.max_row
max_col = ws.max_column
last_col_letter = get_column_letter(max_col)
table_range = f"A1:{last_col_letter}{max_row}"

# Crear la tabla
excel_table = Table(displayName="TablaMaritima", ref=table_range)

# Estilo de la tabla (puedes cambiar el estilo si quieres)
style = TableStyleInfo(
    name="TableStyleMedium9",
    showFirstColumn=False,
    showLastColumn=False,
    showRowStripes=True,
    showColumnStripes=False,
)
excel_table.tableStyleInfo = style

# Añadir la tabla al worksheet
ws.add_table(excel_table)

# Ajustar automáticamente el ancho de columnas
for col in ws.columns:
    max_length = 0
    column = col[0].column_letter  # Letra de la columna (A, B, C...)
    for cell in col:
        try:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        except:
            pass
    adjusted_width = max_length + 2  # Añade algo de espacio extra
    ws.column_dimensions[column].width = adjusted_width
# Guardar el archivo
wb.save(full_path)
print(f"File successfully saved as  '{file_name}' in the directory '{directory}'.")


File successfully saved as  'Maritime_Template_2025-07-25_3.xlsx' in the directory 'c:\Users\SANCHEZ-SERRADORMiri\OneDrive - Horse\Documentos - Horse Supply Chain\Transport & Packaging\Exchange VRAC\02_Engineering Department\00. Transport Projects\Datos cifrados - Quotation Tool\Maritime Tool'.


## Terrestre

### Rate terrestre

In [97]:
renombrar_columnas = {
    "Country": "Pais_Origen",
    "ZIP Code": "ZIP Origen",
    "Name" : "Nombre_Origen",
    "City": "City_Origen",
    "Country.1" : "Pais_Destino",
    "ZIP Code.1" : "Zip_Destino",
    "Name.1" : "Name_Destino",
    "City.1" : "City_Destino"
}
Template_Terrestre = Template_Terrestre.rename(columns=renombrar_columnas)


In [98]:
# MERGE con indicador para detectar combinaciones no encontradas
Template_Terrestre = Template_Terrestre.merge(
    CifradosInland,
    left_on=["City_Origen", "City_Destino"],
    right_on=["Origin_City", "Destination_City"],
    how="left",
    indicator=True
)

# Crear columna de errores con mensaje detallado
Template_Terrestre["Indicador errores"] = Template_Terrestre.apply(
    lambda row: (
        f"Combination '{row['Origin_City']}' + '{row['Destination_City']}' not found in Cifrados Overseas-Inland reference mapping."
        if row["_merge"] != "both" else ""
    ),
    axis=1
)

# Eliminar columna de merge auxiliar
Template_Terrestre.drop(columns=["_merge"], inplace=True)

# Eliminar columnas innecesarias
columns_to_drop = [
    'Picking_country_code', 'Picking_Country', 'Origin_ZIP', 'Origin_City',
    'Delivery_country_code', 'Delivery_Country', 'Destination_ZIP', 'Destination_City',
    'Axis', 'Avg_cost (EUR)', 'Distance (km)', 'Eur/km',
    'Observaciones Avg_cost (EUR)', 'x', 'c'
]

Template_Terrestre = Template_Terrestre.drop(columns=columns_to_drop, errors="ignore")


### Embalaje terrestre

In [99]:
# Hacer el merge
Template_Terrestre = Template_Terrestre.merge(
    Mapping_Motores_reducido,
    left_on=["Part Number (PN)", "Packaging Code"],
    right_on=["Reference", "Packaging Code"],
    how="left",
    indicator=True
)

# Asegurarse que la columna exista
if "Indicador errores" not in Template_Terrestre.columns:
    Template_Terrestre["Indicador errores"] = ""

# Concatenar mensaje por fila, sumando al contenido previo
Template_Terrestre["Indicador errores"] = Template_Terrestre.apply(
    lambda row: (
        str(row["Indicador errores"]) +
        (f"Combination '{row['Part Number (PN)']}' + '{row['Packaging Code']}' not found in Reduced Packaging reference mapping. "
         if row["_merge"] != "both" else "")
    ),
    axis=1
)
# Limpiar columnas auxiliares
Template_Terrestre = Template_Terrestre.drop(columns=["Reference", "_merge"])


### CALCULOS INLAND RATES

In [100]:
Filling_Weight = 24750   # Saturación a 24750 kg
Filling_Rate_Max_Terrestre=85 # Saturación a 85 m^3


Template_Terrestre["Total_Weight"] = (
    Template_Terrestre["Qté / UC"] * Template_Terrestre["Part Weight (kg)"]
    + Template_Terrestre["Weight EMPTY (kg)"]
)

Template_Terrestre["Packaging Volume"] = (Template_Terrestre["Largo"] * Template_Terrestre["Anchura"] * Template_Terrestre["Altura"]) / 1_000_000_000

Template_Terrestre['Part Volume'] = Template_Terrestre['Packaging Volume'] / Template_Terrestre['Qté / UC']


Template_Terrestre["Saturation"] = Template_Terrestre.apply(
    lambda row: "" if pd.isna(row["Total_Weight"]) 
    else "V" if row["Total_Weight"] < Filling_Weight 
    else "W",
    axis=1
)

def calcular_costes_logistica_terrestre(df, filling_rate_max, filling_weight):
    # Aseguramos que los parámetros sean flotantes
    filling_rate_max = float(filling_rate_max)
    filling_weight = float(filling_weight)

    # Convertimos las columnas de tasas a numérico, si ya son nan no las tocará
    df['Costo Ideal (EUR)'] = pd.to_numeric(df['Costo Ideal (EUR)'], errors='coerce')
    df['Part Volume'] = pd.to_numeric(df['Part Volume'], errors='coerce')

    # Creamos las columnas vacías por defecto
    df["LOG €/m3"] = None
    df["LOG €/Part"] = None
    
    # Condición cuando la columna "Saturation" es "V" o "W"
    mask_v = df['Saturation'] == 'V'
    mask_w = df['Saturation'] == 'W'

    # Si "Saturation" es 'V', usamos Filling_Rate_Max
    df.loc[mask_v, "LOG €/m3"] = df.loc[mask_v, "Costo Ideal (EUR)"] / filling_rate_max
    df.loc[mask_v, "LOG €/Part"] = df.loc[mask_v, "LOG €/m3"] * df.loc[mask_v, "Part Volume"]


    # Si "Saturation" es 'W', usamos Filling_Weight

    Volumen_maximo = (Filling_Weight / Template_Terrestre["Total_Weight"] ) * Template_Terrestre["Packaging Volume"]
    
    df.loc[mask_w, "LOG €/m3"] = df.loc[mask_w, "Costo Ideal (EUR)"] / Volumen_maximo
    df.loc[mask_w, "LOG €/Part"] = df.loc[mask_w, "LOG €/m3"] * df.loc[mask_w, "Part Volume"]


    return df

# Si la columna 'Saturation' tiene valores 'V' o 'W', aplicamos la función correspondiente
if 'Saturation' in Template_Terrestre.columns:
    # Si 'Saturation' tiene 'V', usamos Filling_Rate_Max, y si tiene 'W', usamos Volumen_maximo
    Template_Terrestre = calcular_costes_logistica_terrestre(Template_Terrestre, Filling_Rate_Max_Terrestre, Filling_Weight)


### AÑADIR VTT

In [101]:
# Hacer el left join para mantener todas las filas de Template_Maritimo
Template_Terrestre = Template_Terrestre.merge(
    VTT_Final,
    left_on=["Pais_Origen", "ZIP Origen", "Pais_Destino", "Zip_Destino"],
    right_on=["Picking_Country", "ZIP Code", "Delivery_Country", "ZIP Code.1"],
    how="left",
    indicator=True  # Nos indica si hizo match o no
)

# Asegurarse de que la columna "Indicador errores" ya existe (o crearla vacía si no)
if "Indicador errores" not in Template_Terrestre.columns:
    Template_Terrestre["Indicador errores"] = ""

# Añadir mensaje solo si no hizo match
Template_Terrestre["Indicador errores"] = Template_Terrestre.apply(
    lambda row: (
        (row["Indicador errores"] + " | " if row["Indicador errores"] else "") +
        f"Route from {row['Pais_Origen']} {row['ZIP Origen']} to {row['Pais_Destino']} {row['Zip_Destino']} not found in VTT"
    ) if row["_merge"] != "both" else row["Indicador errores"],
    axis=1
)

# Eliminar columna de tracking del merge
Template_Terrestre = Template_Terrestre.drop(columns=["_merge"])

Template_Terrestre = Template_Terrestre.drop(columns=['transportType', 'childPlantPublicId', 'Picking_Country',
       'ZIP Code', 'childPlantName', 'parentPlantPublicId', 'Delivery_Country',
       'ZIP Code.1', 'parentPlantName', 'leadTime',
       'nbFlows'])

### CALCULOS FINALES

In [102]:
# Floating Stock €/Part
Template_Terrestre['Floating Stock €/Part'] = (Template_Terrestre['Unit cost (€)'] * [Interes_Financiero] /365) * Template_Terrestre['LeadTime (days)']

# TOTAL €/Part
Template_Terrestre['TOTAL €/Part'] = Template_Terrestre['LOG €/Part'] + Template_Terrestre['Unit cost (€)'] + Template_Terrestre['Floating Stock €/Part']

# Annual TO
Template_Terrestre['Annual weight'] = (Template_Terrestre['TOTAL €/Part'] * Template_Terrestre['Anual Needs']) / 1000

# FCF Pipe
Template_Terrestre['FCF Pipe'] = (Template_Terrestre['Daily Need'] * Template_Terrestre['Unit cost (€)'] * Template_Terrestre['LeadTime (days)']) / 1000


### LIMPIEZA FINAL

In [103]:
Template_Terrestre.columns
Template_Terrestre = Template_Terrestre.drop_duplicates()

In [104]:
col_a_mover = "Indicador errores"
columnas = [col_a_mover] + [col for col in Template_Terrestre.columns if col != col_a_mover] 
Template_Terrestre = Template_Terrestre[columnas]

In [105]:
renombrar_columnas = {
    "Indicador errores": "Error Indicator",
    "Nombre_Origen": "Name of Origin",
    "Pais_Origen" : "Country of Origin",
    "City_Origen": "City of Origin",
    "ZIP Origen" : "Origin ZIP",
    "Name_Destino" : "Name of Destination",
    "Pais_Destino" : "Country of Destination",
    "City_Destino" : "City of Destination",
    "Zip_Destino" : "Destination ZIP"
}


In [106]:
Template_Terrestre = Template_Terrestre.rename(columns=renombrar_columnas)

In [107]:
Template_Terrestre = Template_Terrestre.drop(columns=['POL', 'POD', 'Maritimo'])

In [108]:
for col in Template_Terrestre.columns:
    if col != 'Daily Need':
        try:
            Template_Terrestre[col] = pd.to_numeric(Template_Terrestre[col], errors='ignore')
        except:
            pass  # En caso de error, dejar como está

# Redondear columnas numéricas (excepto 'Daily Need') a 2 decimales
cols_to_round = Template_Terrestre.select_dtypes(include=[np.number]).columns.drop('Daily Need', errors='ignore')
Template_Terrestre[cols_to_round] = Template_Terrestre[cols_to_round].round(2)

# Truncar 'Daily Need' sin afectar NaN
Template_Terrestre['Daily Need'] = Template_Terrestre['Daily Need'].apply(
    lambda x: int(x) if pd.notna(x) else np.nan
)

C:\Users\SANCHEZ-SERRADORMiri\AppData\Local\Temp\ipykernel_17584\4210235572.py:4: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  Template_Terrestre[col] = pd.to_numeric(Template_Terrestre[col], errors='ignore')


### EXPORT A EXCEL

In [109]:
# Ruta del directorio
folder_terrestre = "Land Tool"
directory = os.path.join(base_path, folder_terrestre)

# Inicializa un contador
counter = 1

# Obtener la fecha actual en formato YYYY-MM-DD
current_date = datetime.now().strftime('%Y-%m-%d')

# Crear el nombre del archivo con la fecha
file_name = f"Land_Template_{current_date}_{counter}.xlsx"

# Asegurar que el archivo no existe, incrementar el contador
while os.path.exists(os.path.join(directory, file_name)):
    counter += 1
    file_name = f"Land_Template_{current_date}_{counter}.xlsx"

# Ruta completa del archivo
full_path = os.path.join(directory, file_name)

# Guardar el DataFrame como Excel
Template_Terrestre.to_excel(full_path, index=False, sheet_name="Hoja1")

# --- FORMATEAR COMO TABLA ---
# Cargar el archivo con openpyxl
wb = load_workbook(full_path)
ws = wb["Hoja1"]

# Obtener el rango de la tabla
max_row = ws.max_row
max_col = ws.max_column
last_col_letter = get_column_letter(max_col)
table_range = f"A1:{last_col_letter}{max_row}"

# Crear la tabla
excel_table = Table(displayName="TablaTerrestre", ref=table_range)

# Estilo de la tabla (puedes cambiarlo si quieres)
style = TableStyleInfo(
    name="TableStyleMedium9",
    showFirstColumn=False,
    showLastColumn=False,
    showRowStripes=True,
    showColumnStripes=False,
)
excel_table.tableStyleInfo = style

# Añadir la tabla al worksheet
ws.add_table(excel_table)

# Ajustar automáticamente el ancho de columnas
for col in ws.columns:
    max_length = 0
    column = col[0].column_letter  # Letra de la columna (A, B, C...)
    for cell in col:
        try:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        except:
            pass
    adjusted_width = max_length + 2
    ws.column_dimensions[column].width = adjusted_width

# Guardar el archivo
wb.save(full_path)
print(f"File successfully saved as  '{file_name}' in the directory '{directory}'.")


File successfully saved as  'Land_Template_2025-07-25_3.xlsx' in the directory 'c:\Users\SANCHEZ-SERRADORMiri\OneDrive - Horse\Documentos - Horse Supply Chain\Transport & Packaging\Exchange VRAC\02_Engineering Department\00. Transport Projects\Datos cifrados - Quotation Tool\Land Tool'.
